<a href="https://colab.research.google.com/github/deepika1715/Artificial-Intelligence/blob/main/Detecting_deepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install keras
!pip install efficientnet
!pip install kaggle

In [ ]:
from google.colab import files

# Upload kaggle.json
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"deepika1715","key":"9e13b3074137a4ad61588a8112ef7c5d"}'}

In [ ]:
#making .kaggle directory under root
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
#moving kaggle.json to .kaggle directory
!mv kaggle.json ~/.kaggle/

In [ ]:
#changing access permision for the json file
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import kaggle

In [ ]:
#importing required libraries
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import scikitplot as skplt

In [ ]:
#downloading the dataset
#!kaggle datasets download -d fakecatcherai/dfdc-dataset
!kaggle competitions download -c deepfake-detection-challenge




100% 4.12G/4.13G [00:43<00:00, 253MB/s]
100% 4.13G/4.13G [00:43<00:00, 102MB/s]


In [19]:
!unzip /content/deepfake-detection-challenge.zip -d /content/DFDC/


Archive:  /content/deepfake-detection-challenge.zip
  inflating: /content/DFDC/sample_submission.csv  
  inflating: /content/DFDC/test_videos/aassnaulhq.mp4  
  inflating: /content/DFDC/test_videos/aayfryxljh.mp4  
  inflating: /content/DFDC/test_videos/acazlolrpz.mp4  
  inflating: /content/DFDC/test_videos/adohdulfwb.mp4  
  inflating: /content/DFDC/test_videos/ahjnxtiamx.mp4  
  inflating: /content/DFDC/test_videos/ajiyrjfyzp.mp4  
  inflating: /content/DFDC/test_videos/aktnlyqpah.mp4  
  inflating: /content/DFDC/test_videos/alrtntfxtd.mp4  
  inflating: /content/DFDC/test_videos/aomqqjipcp.mp4  
  inflating: /content/DFDC/test_videos/apedduehoy.mp4  
  inflating: /content/DFDC/test_videos/apvzjkvnwn.mp4  
  inflating: /content/DFDC/test_videos/aqrsylrzgi.mp4  
  inflating: /content/DFDC/test_videos/axfhbpkdlc.mp4  
  inflating: /content/DFDC/test_videos/ayipraspbn.mp4  
  inflating: /content/DFDC/test_videos/bcbqxhziqz.mp4  
  inflating: /content/DFDC/test_videos/bcvheslzrq.mp4  
 

In [20]:

import cv2
import os
import json

def extract_frames(video_path, output_folder, frame_rate=1):
    video_capture = cv2.VideoCapture(video_path)
    count = 0
    success, image = video_capture.read()
    while success:
        if count % frame_rate == 0:
            cv2.imwrite(os.path.join(output_folder, f"frame_{count}.jpg"), image)
        success, image = video_capture.read()
        count += 1
    video_capture.release()

# Example usage:
extract_frames('/content/DFDC/test_videos', '/content/DFDC/test_frames')




In [22]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory structure
train_dir = '/content/DFDC/train_frames'
val_dir = '/content/DFDC/train_sample_videos'

# Example data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [23]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# Load EfficientNetB0 with pre-trained weights on ImageNet and exclude the top layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)  # Adding Dropout for regularization
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

# Combine base model and custom layers
model = Model(inputs=base_model.input, outputs=predictions)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [24]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


In [28]:
#code for splitting data into 80% training and 20% validation

import shutil
from sklearn.model_selection import train_test_split

# Paths to dataset directories
data_dir = "/content/DFDC/train_sample_videos"
train_dir = "/content/DFDC/train_videos"
val_dir = "/content/DFDC/validation_videos"

# Load the metadata file that contains labels
metadata_path = os.path.join(data_dir, 'metadata.json')
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

# Create lists for real and fake videos based on metadata
real_videos = [video for video, info in metadata.items() if info['label'] == 'REAL']
fake_videos = [video for video, info in metadata.items() if info['label'] == 'FAKE']

# Split the videos into train and validation sets
train_real, val_real = train_test_split(real_videos, test_size=0.2, random_state=42)
train_fake, val_fake = train_test_split(fake_videos, test_size=0.2, random_state=42)

# Function to move files
def move_files(video_list, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)
    for video in video_list:
        src_path = os.path.join(data_dir, video)
        dest_path = os.path.join(dest_dir, video)
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_path)

# Create real and fake folders inside train and validation directories
os.makedirs(os.path.join(train_dir, 'real'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'fake'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'real'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'fake'), exist_ok=True)

# Move the files
move_files(train_real, os.path.join(train_dir, 'real'))
move_files(train_fake, os.path.join(train_dir, 'fake'))
move_files(val_real, os.path.join(val_dir, 'real'))
move_files(val_fake, os.path.join(val_dir, 'fake'))


In [29]:
# extracting frames from videos for training data
import cv2

# Paths to the 'real' and 'fake' directories containing videos
train_real_video_dir = '/content/DFDC/train_videos/real'
train_fake_video_dir = '/content/DFDC/train_videos/fake'

# Paths to save the extracted frames
train_real_frames_dir = '/content/DFDC/train_frames/real'
train_fake_frames_dir = '/content/DFDC/train_frames/fake'

# Function to extract frames from a video and save them to a directory
def extract_frames(video_path, output_dir, frame_rate=1):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Open the video file
    video_cap = cv2.VideoCapture(video_path)
    count = 0
    success = True
    fps = video_cap.get(cv2.CAP_PROP_FPS)
    interval = int(fps // frame_rate)  # Calculate interval to capture frames

    while success:
        success, frame = video_cap.read()
        if success and count % interval == 0:
            # Save the frame as an image file
            frame_filename = os.path.join(output_dir, f"frame_{count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
        count += 1

    # Release the video capture object
    video_cap.release()

# Function to process all videos in a directory
def process_videos_in_directory(video_dir, frames_dir):
    for video_name in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_name)
        video_output_dir = os.path.join(frames_dir, os.path.splitext(video_name)[0])

        # Extract frames from the video
        extract_frames(video_path, video_output_dir)

# Extract frames from 'real' videos
process_videos_in_directory(train_real_video_dir, train_real_frames_dir)

# Extract frames from 'fake' videos
process_videos_in_directory(train_fake_video_dir, train_fake_frames_dir)

print("Train frames extraction completed.")


Frame extraction completed.


In [1]:

import json
from sklearn.model_selection import train_test_split

# Paths to dataset directories
test_data_dir = "/content/DFDC/test_videos"


# Load the metadata file that contains labels
metadata_path = os.path.join(data_dir, 'metadata.json')
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

# Create lists for test videos based on metadata
real_videos = [video for video, info in metadata.items() if info['label'] == 'REAL']
fake_videos = [video for video, info in metadata.items() if info['label'] == 'FAKE']

# Split the videos into test sets
test_real = test_data_split(real_videos, test_size=0.2, random_state=42)[0]  # Keep only the training part
test_fake = test_data_split(fake_videos, test_size=0.2, random_state=42)[0]  # Keep only the training part

# Function to move files
def move_files(video_list, src_dir, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)
    for video in video_list:
        src_path = os.path.join(src_dir, video)
        dest_path = os.path.join(dest_dir, video)
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_path)
        else:
            print(f"Warning: {src_path} does not exist.")

# Create real and fake folders inside test directories
os.makedirs(os.path.join(test_data_dir, 'real'), exist_ok=True)
os.makedirs(os.path.join(test_data_dir, 'fake'), exist_ok=True)


# Move the files from the original data directory to the test directories
move_files(test_real, data_dir, os.path.join(test_data_dir, 'real'))
move_files(test_fake, data_dir, os.path.join(test_data_dir, 'fake'))



NameError: name 'os' is not defined

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory structure - Verify that these directories exist and contain images
train_dir = '/root/kaggle/DFDC/train_frames'
val_dir = '/root/kaggle/DFDC/train_sample_videos'

# Print the number of images found in each directory
print("Number of images in training directory:", len(os.listdir(train_dir)))
print("Number of images in validation directory:", len(os.listdir(val_dir)))

# Example data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Check if generators are yielding data
print("Training generator yields data:", bool(next(train_generator, None)))
print("Validation generator yields data:", bool(next(validation_generator, None)))

Number of images in training directory: 2
Number of images in validation directory: 401
Found 0 images belonging to 2 classes.
Found 0 images belonging to 0 classes.
Training generator yields data: True
Validation generator yields data: True


In [30]:
# extracting frames from videos for validation data
import cv2

# Paths to the 'real' and 'fake' directories containing videos
validation_real_video_dir = '/content/DFDC/validation_videos/real'
validation_fake_video_dir = '/content/DFDC/validation_videos/fake'

# Paths to save the extracted frames
validation_real_frames_dir = '/content/DFDC/validation_frames/real'
validation_fake_frames_dir = '/content/DFDC/validation_frames/fake'

# Function to extract frames from a video and save them to a directory
def extract_frames(video_path, output_dir, frame_rate=1):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Open the video file
    video_cap = cv2.VideoCapture(video_path)
    count = 0
    success = True
    fps = video_cap.get(cv2.CAP_PROP_FPS)
    interval = int(fps // frame_rate)  # Calculate interval to capture frames

    while success:
        success, frame = video_cap.read()
        if success and count % interval == 0:
            # Save the frame as an image file
            frame_filename = os.path.join(output_dir, f"frame_{count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
        count += 1

    # Release the video capture object
    video_cap.release()

# Function to process all videos in a directory
def process_videos_in_directory(video_dir, frames_dir):
    for video_name in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_name)
        video_output_dir = os.path.join(frames_dir, os.path.splitext(video_name)[0])

        # Extract frames from the video
        extract_frames(video_path, video_output_dir)

# Extract frames from 'real' videos
process_videos_in_directory(validation_real_video_dir, validation_real_frames_dir)

# Extract frames from 'fake' videos
process_videos_in_directory(validation_fake_video_dir, validation_fake_frames_dir)

print("Validation frames extraction completed.")

Validation frames extraction completed.


In [31]:
#extracting test frames


def extract_frames_from_videos(video_dir, output_dir, frame_rate=1):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # List all video files in the video_dir
    video_files = [f for f in os.listdir(video_dir) if os.path.isfile(os.path.join(video_dir, f))]

    for video_file in video_files:
        video_path = os.path.join(video_dir, video_file)
        output_folder = os.path.join(output_dir, os.path.splitext(video_file)[0])  # Create a folder for each video
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        video_capture = cv2.VideoCapture(video_path)
        count = 0
        success, image = video_capture.read()
        while success:
            if count % frame_rate == 0:
                cv2.imwrite(os.path.join(output_folder, f"frame_{count}.jpg"), image)
            success, image = video_capture.read()
            count += 1
        video_capture.release()
        print(f'Frames extracted from {video_file} to {output_folder}')

# Example usage:
extract_frames_from_videos('/content/DFDC/test_videos', '/content/DFDC/test_frames')



FileNotFoundError: [Errno 2] No such file or directory: '/root/kaggle/DFDC/test_videos'

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory structure - Verify that these directories exist and contain images
train_dir = '/root/kaggle/DFDC/train_frames'
val_dir = '/root/kaggle/DFDC/train_sample_videos'
test_dir = '/root/kaggle/DFDC/test_frames' # Assuming you have a test directory

# ... (rest of the code remains the same)

# Create a test generator
test_datagen = ImageDataGenerator(rescale=1./255) # You might

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory containing test data
test_dir = '/root/kaggle/DFDC/test_frames'

# Create a data generator for test data
test_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for testing

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Assuming binary classification
    shuffle=False  # Important for evaluation to ensure predictions correspond to the correct labels
)


Found 13809 images belonging to 47 classes.


In [ ]:
# Verify test_generator
print(test_generator.class_indices)  # Check class indices
print(len(test_generator.filenames))  # Check number of test samples


{'aktnlyqpah': 0, 'apedduehoy': 1, 'bkuzquigyt': 2, 'blnmxntbey': 3, 'bofrwgeyjo': 4, 'cnxccbjlct': 5, 'etdliwticv': 6, 'frqfsucgao': 7, 'gfcycflhbo': 8, 'gfgcwxkbjd': 9, 'gpsxfxrjrr': 10, 'hxwtsaydal': 11, 'iorbtaarte': 12, 'jhczqfefgw': 13, 'jsysgmycsx': 14, 'jyoxdvxpza': 15, 'kcjvhgvhpt': 16, 'kmcdjxmnoa': 17, 'kvmpmhdxly': 18, 'lebzjtusnr': 19, 'ljauauuyka': 20, 'mdfndlljvt': 21, 'ncmpqwmnzb': 22, 'nxgzmgzkfv': 23, 'pxcfrszlgi': 24, 'qcbkztamqc': 25, 'qlvsqdroqo': 26, 'qsjiypnjwi': 27, 'ryxaqpfubf': 28, 'sylnrepacf': 29, 'tejfudfgpq': 30, 'tyjpjpglgx': 31, 'udxqbhgvvx': 32, 'usqqvxcjmg': 33, 'vizerpsvbz': 34, 'vokrpfjpeb': 35, 'vssmlqoiti': 36, 'wcqvzujamg': 37, 'wixbuuzygv': 38, 'wvgviwnwob': 39, 'xdezcezszc': 40, 'xphdfgmfmz': 41, 'ybbrkacebd': 42, 'yllztsrwjw': 43, 'ystdtnetgj': 44, 'zfobicuigx': 45, 'zgbhzkditd': 46}
13809


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Found 13809 images belonging to 47 classes.


In [ ]:
# Evaluate on a batch from the generator
x_batch, y_batch = next(test_generator)
test_loss, test_accuracy = model.evaluate(x_batch, y_batch)
print(f'Test loss: {test_loss:.2f}, Test accuracy: {test_accuracy:.2f}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.0000e+00 - loss: 0.6933
Test loss: 0.69, Test accuracy: 0.00


In [ ]:
# Verify if test_generator is empty
if len(test_generator.filenames) == 0:
    print("Error: No test samples found in the directory.")
else:
    # Evaluate on a batch from the generator
    x_batch, y_batch = next(test_generator)
    test_loss, test_accuracy = model.evaluate(x_batch, y_batch)
    print(f'Test loss: {test_loss:.2f}, Test accuracy: {test_accuracy:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 0.6933
Test loss: 0.69, Test accuracy: 0.00


In [26]:
import shutil # Import the shutil module
folder_path = '/content/DFDC/train_frames'
shutil.rmtree(folder_path)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test loss: {test_loss:.2f}, Test accuracy: {test_accuracy:.2f}')

# Predict on test data
test_generator.reset()
predictions = model.predict(test_generator)
y_pred = (predictions > 0.5).astype(int)

# Get true labels
y_true = test_generator.classes

# Compute confusion matrix and classification report
cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys())

print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(cr)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Plot accuracy and loss curves
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss')

plt.show()


In [ ]:
train_dir = '/content/DFDC/train_frames'
val_dir = 'path_to_val_dir'
test_dir = 'path_to_test_dir'

# Image data generator for data augmentation
datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                             height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                             horizontal_flip=True, fill_mode='nearest')

# Load the train and validation data
train_generator = datagen.flow_from_directory(train_dir, target_size=(224, 224),
                                              batch_size=32, class_mode='binary')
val_generator = datagen.flow_from_directory(val_dir, target_size=(224, 224),
                                            batch_size=32, class_mode='binary')
test_generator = datagen.flow_from_directory(test_dir, target_size=(224, 224),
                                             batch_size=32, class_mode='binary', shuffle=False)

# Build the model using EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Adding dropout for regularization
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])

# Train the model
model.fit(train_generator, epochs=20, validation_data=val_generator)

# Evaluate the model on test data
test_loss, test_accuracy, test_precision, test_recall, test_auc = model.evaluate(test_generator)
y_pred = model.predict(test_generator)
y_true = test_generator.classes

# Calculate F1-score
f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)

# Print out the results
print(f"Accuracy: {test_accuracy * 100:.2f}%")
print(f"Precision: {test_precision * 100:.2f}%")
print(f"Recall: {test_recall * 100:.2f}%")
print(f"F1-score: {f1_score * 100:.2f}%")
print(f"AUC-ROC: {test_auc:.2f}")

# For a detailed classification report
print(classification_report(y_true, (y_pred > 0.5).astype(int)))

# AUC-ROC score using sklearn
auc_roc = roc_auc_score(y_true, y_pred)
print(f"AUC-ROC (sklearn): {auc_roc:.2f}")